# FizzBuzz
从1开始，到3的倍数就说fizz，到5的倍数就说buzz，到15的倍数就说fizzbuzz

In [1]:
def fizz_buzz_encode(i):
    if i % 15 == 0: return 3
    elif i % 5 == 0: return 2
    elif i % 3 == 0: return 1
    else: return 0

def fizz_buzz_decode(i,prediction):
    return [str(i),"fizz","buzz","fizzbuzz"][prediction]

def helper(i):
    print(fizz_buzz_decode(i,fizz_buzz_encode(i)))

for i in range(1,17):
    helper(i)

1
2
fizz
4
buzz
fizz
7
8
fizz
buzz
11
fizz
13
14
fizzbuzz
16


In [2]:
import numpy as np
import torch

In [3]:
NUM_DIGITS = 10

#将数据变成二进制更加易于计算
def binary_encode(i,num_digits):
    return np.array([i >> d & 1 for d in range(num_digits)])[::-1]
# binary_encode(31,NUM_DIGITS)
trX = torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(101,2 ** NUM_DIGITS)])
#这里Y是表示类别，而X已经是float了，为了在下面不出现数据类型错误，要将Y改为LongTensor
trY = torch.LongTensor([fizz_buzz_encode(i) for i in range(101,2 ** NUM_DIGITS)])

然后用pytorch定义模型

In [9]:
class TwoLayerModule(torch.nn.Module):
    def __init__(self):
        super(TwoLayerModule,self).__init__()
        self.linear1 = torch.nn.Linear(10,100)
        self.linear2 = torch.nn.Linear(100,4)
    def forward(self,trX):
        y_pred = self.linear2(self.linear1(trX).clamp(min=0))
        return y_pred

In [10]:
NUM_HIDDEN = 100

# model = torch.nn.Sequential(
#     torch.nn.Linear(NUM_DIGITS,NUM_HIDDEN),
#     torch.nn.ReLU(),
#     torch.nn.Linear(NUM_HIDDEN,4)# 4 logits, after softmax,we get a probability distribution
# )
model = TwoLayerModule()
if torch.cuda.is_available():
    model = model.cuda()

In [11]:
loss_fn = torch.nn.CrossEntropyLoss() #CrossEntropyLoss()函数本质上是用来评估两个分布的相似度
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)

以下是模型的训练代码

In [12]:
BATCH_SIZE = 128
for epoch in range(1000):
    for start in range(0,len(trX),BATCH_SIZE):
        end = start + BATCH_SIZE
        batchX = trX[start:end]
        batchY = trY[start:end]
        if torch.cuda.is_available():
            batchX = batchX.cuda()
            batchY = batchY.cuda()
            
        y_pred = model(batchX)
        loss = loss_fn(y_pred,batchY)
        print("Epoch:",epoch,loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()     

Epoch: 0 1.4261661767959595
Epoch: 0 1.4149421453475952
Epoch: 0 1.4079960584640503
Epoch: 0 1.3976151943206787
Epoch: 0 1.3913843631744385
Epoch: 0 1.379510760307312
Epoch: 0 1.373518943786621
Epoch: 0 1.3254834413528442
Epoch: 1 1.3151131868362427
Epoch: 1 1.3040704727172852
Epoch: 1 1.286068320274353
Epoch: 1 1.2982124090194702
Epoch: 1 1.2867891788482666
Epoch: 1 1.2713888883590698
Epoch: 1 1.2636934518814087
Epoch: 1 1.2012624740600586
Epoch: 2 1.2393035888671875
Epoch: 2 1.2306084632873535
Epoch: 2 1.207905650138855
Epoch: 2 1.2326072454452515
Epoch: 2 1.2183663845062256
Epoch: 2 1.202207088470459
Epoch: 2 1.198484182357788
Epoch: 2 1.1326961517333984
Epoch: 3 1.1912407875061035
Epoch: 3 1.186876654624939
Epoch: 3 1.164541482925415
Epoch: 3 1.1943919658660889
Epoch: 3 1.1789458990097046
Epoch: 3 1.1643037796020508
Epoch: 3 1.1681849956512451
Epoch: 3 1.1067217588424683
Epoch: 4 1.1649976968765259
Epoch: 4 1.1654167175292969
Epoch: 4 1.1460527181625366
Epoch: 4 1.1757651567459106


Epoch: 41 1.1390584707260132
Epoch: 41 1.044658899307251
Epoch: 42 1.1077741384506226
Epoch: 42 1.122043251991272
Epoch: 42 1.1100387573242188
Epoch: 42 1.1284106969833374
Epoch: 42 1.1190407276153564
Epoch: 42 1.1139628887176514
Epoch: 42 1.1382606029510498
Epoch: 42 1.0423285961151123
Epoch: 43 1.1058067083358765
Epoch: 43 1.120859980583191
Epoch: 43 1.108725666999817
Epoch: 43 1.1271824836730957
Epoch: 43 1.1177420616149902
Epoch: 43 1.1127336025238037
Epoch: 43 1.137467384338379
Epoch: 43 1.0396908521652222
Epoch: 44 1.1035598516464233
Epoch: 44 1.119799017906189
Epoch: 44 1.1074897050857544
Epoch: 44 1.1255863904953003
Epoch: 44 1.1161959171295166
Epoch: 44 1.1114157438278198
Epoch: 44 1.1363619565963745
Epoch: 44 1.0372170209884644
Epoch: 45 1.1012059450149536
Epoch: 45 1.1182465553283691
Epoch: 45 1.1058415174484253
Epoch: 45 1.1238967180252075
Epoch: 45 1.1146976947784424
Epoch: 45 1.1098459959030151
Epoch: 45 1.1351706981658936
Epoch: 45 1.034450888633728
Epoch: 46 1.099342823

Epoch: 78 1.0105232000350952
Epoch: 78 1.0193015336990356
Epoch: 78 1.0368760824203491
Epoch: 78 1.0234711170196533
Epoch: 78 1.061553716659546
Epoch: 78 0.905424952507019
Epoch: 79 0.9762681722640991
Epoch: 79 1.0415806770324707
Epoch: 79 1.007086157798767
Epoch: 79 1.0154318809509277
Epoch: 79 1.0338759422302246
Epoch: 79 1.0202417373657227
Epoch: 79 1.0591920614242554
Epoch: 79 0.9002841114997864
Epoch: 80 0.9720954895019531
Epoch: 80 1.038677453994751
Epoch: 80 1.0034008026123047
Epoch: 80 1.011905312538147
Epoch: 80 1.0308327674865723
Epoch: 80 1.017438292503357
Epoch: 80 1.0563610792160034
Epoch: 80 0.8962867259979248
Epoch: 81 0.9680212140083313
Epoch: 81 1.036115050315857
Epoch: 81 1.0005698204040527
Epoch: 81 1.0090031623840332
Epoch: 81 1.0290111303329468
Epoch: 81 1.0149654150009155
Epoch: 81 1.0538214445114136
Epoch: 81 0.8924326300621033
Epoch: 82 0.9641902446746826
Epoch: 82 1.0341906547546387
Epoch: 82 0.998535692691803
Epoch: 82 1.0060935020446777
Epoch: 82 1.0254503488

Epoch: 114 0.8844535946846008
Epoch: 114 0.8910751938819885
Epoch: 114 0.9145820736885071
Epoch: 114 0.910439670085907
Epoch: 114 0.9571449756622314
Epoch: 114 0.7448096871376038
Epoch: 115 0.8246095180511475
Epoch: 115 0.9239889979362488
Epoch: 115 0.8804707527160645
Epoch: 115 0.887079656124115
Epoch: 115 0.9106419086456299
Epoch: 115 0.9061648845672607
Epoch: 115 0.9538341164588928
Epoch: 115 0.7407636046409607
Epoch: 116 0.8205500841140747
Epoch: 116 0.9206118583679199
Epoch: 116 0.8760089874267578
Epoch: 116 0.8834913969039917
Epoch: 116 0.9072474837303162
Epoch: 116 0.9029657244682312
Epoch: 116 0.9504417777061462
Epoch: 116 0.7359363436698914
Epoch: 117 0.8163052797317505
Epoch: 117 0.9161262512207031
Epoch: 117 0.8724145293235779
Epoch: 117 0.8795612454414368
Epoch: 117 0.9031935930252075
Epoch: 117 0.8994414210319519
Epoch: 117 0.9473652243614197
Epoch: 117 0.7320218086242676
Epoch: 118 0.811917245388031
Epoch: 118 0.9124950170516968
Epoch: 118 0.8685227036476135
Epoch: 118 0.

Epoch: 150 0.5974173545837402
Epoch: 151 0.6794559955596924
Epoch: 151 0.7948349714279175
Epoch: 151 0.738721489906311
Epoch: 151 0.7469464540481567
Epoch: 151 0.7767113447189331
Epoch: 151 0.787114679813385
Epoch: 151 0.8283246159553528
Epoch: 151 0.593004584312439
Epoch: 152 0.6755030155181885
Epoch: 152 0.7907510995864868
Epoch: 152 0.7346479296684265
Epoch: 152 0.7439581751823425
Epoch: 152 0.7726508975028992
Epoch: 152 0.783562183380127
Epoch: 152 0.8244619965553284
Epoch: 152 0.588985025882721
Epoch: 153 0.671902060508728
Epoch: 153 0.7871652245521545
Epoch: 153 0.7315226793289185
Epoch: 153 0.7397302985191345
Epoch: 153 0.768278956413269
Epoch: 153 0.780569314956665
Epoch: 153 0.8206432461738586
Epoch: 153 0.5853971838951111
Epoch: 154 0.6678688526153564
Epoch: 154 0.7848404049873352
Epoch: 154 0.7275079488754272
Epoch: 154 0.7360872030258179
Epoch: 154 0.7651487588882446
Epoch: 154 0.777188241481781
Epoch: 154 0.8182687759399414
Epoch: 154 0.5820592641830444
Epoch: 155 0.663175

...

Epoch: 823 0.046710215508937836
Epoch: 823 0.08263102173805237
Epoch: 823 0.05290363356471062
Epoch: 823 0.06090155243873596
Epoch: 823 0.024246683344244957
Epoch: 824 0.044897593557834625
Epoch: 824 0.048456668853759766
Epoch: 824 0.03602597117424011
Epoch: 824 0.046539951115846634
Epoch: 824 0.08272277563810349
Epoch: 824 0.05228792876005173
Epoch: 824 0.06056290864944458
Epoch: 824 0.024038350209593773
Epoch: 825 0.04452819004654884
Epoch: 825 0.048435959964990616
Epoch: 825 0.03599526360630989
Epoch: 825 0.046522174030542374
Epoch: 825 0.08212534338235855
Epoch: 825 0.052222222089767456
Epoch: 825 0.060252536088228226
Epoch: 825 0.02467542141675949
Epoch: 826 0.04429307207465172
Epoch: 826 0.04843078553676605
Epoch: 826 0.035810437053442
Epoch: 826 0.046325258910655975
Epoch: 826 0.08177575469017029
Epoch: 826 0.05238444358110428
Epoch: 826 0.06005576252937317
Epoch: 826 0.02423233538866043
Epoch: 827 0.04445619881153107
Epoch: 827 0.04835110902786255
Epoch: 827 0.03575779125094414

Epoch: 857 0.04079273343086243
Epoch: 857 0.043877653777599335
Epoch: 857 0.032428666949272156
Epoch: 857 0.04161689803004265
Epoch: 857 0.07404434680938721
Epoch: 857 0.04735409840941429
Epoch: 857 0.05548682063817978
Epoch: 857 0.021424364298582077
Epoch: 858 0.04043298214673996
Epoch: 858 0.0433921292424202
Epoch: 858 0.03224966675043106
Epoch: 858 0.04165835306048393
Epoch: 858 0.07414928078651428
Epoch: 858 0.0468711256980896
Epoch: 858 0.05457990989089012
Epoch: 858 0.021700939163565636
Epoch: 859 0.04015766829252243
Epoch: 859 0.043349236249923706
Epoch: 859 0.032217856496572495
Epoch: 859 0.04186452552676201
Epoch: 859 0.0741497129201889
Epoch: 859 0.046866703778505325
Epoch: 859 0.05423252284526825
Epoch: 859 0.02223275415599346
Epoch: 860 0.04040348902344704
Epoch: 860 0.04345224052667618
Epoch: 860 0.03199582174420357
Epoch: 860 0.041245318949222565
Epoch: 860 0.07308968901634216
Epoch: 860 0.047094687819480896
Epoch: 860 0.05472898855805397
Epoch: 860 0.021264269948005676
E

Epoch: 893 0.04220720753073692
Epoch: 893 0.04983379691839218
Epoch: 893 0.01890137419104576
Epoch: 894 0.036394067108631134
Epoch: 894 0.038779422640800476
Epoch: 894 0.028673360124230385
Epoch: 894 0.036994967609643936
Epoch: 894 0.06650999188423157
Epoch: 894 0.04197050258517265
Epoch: 894 0.04887443780899048
Epoch: 894 0.019339799880981445
Epoch: 895 0.03608012944459915
Epoch: 895 0.03872146084904671
Epoch: 895 0.02871539257466793
Epoch: 895 0.037434838712215424
Epoch: 895 0.06683572381734848
Epoch: 895 0.042061932384967804
Epoch: 895 0.04870596528053284
Epoch: 895 0.01971297711133957
Epoch: 896 0.03658640384674072
Epoch: 896 0.03883151337504387
Epoch: 896 0.02854248322546482
Epoch: 896 0.036760155111551285
Epoch: 896 0.06577835977077484
Epoch: 896 0.041862793266773224
Epoch: 896 0.04930136352777481
Epoch: 896 0.018743285909295082
Epoch: 897 0.03611672669649124
Epoch: 897 0.03832695633172989
Epoch: 897 0.028368327766656876
Epoch: 897 0.036486100405454636
Epoch: 897 0.06596474349498

Epoch: 929 0.06000816076993942
Epoch: 929 0.03754325956106186
Epoch: 929 0.04476340487599373
Epoch: 929 0.016691913828253746
Epoch: 930 0.03291066735982895
Epoch: 930 0.0347713977098465
Epoch: 930 0.025683343410491943
Epoch: 930 0.03307661414146423
Epoch: 930 0.0600922629237175
Epoch: 930 0.03736073151230812
Epoch: 930 0.043910492211580276
Epoch: 930 0.01702122390270233
Epoch: 931 0.032539062201976776
Epoch: 931 0.03454142063856125
Epoch: 931 0.025619562715291977
Epoch: 931 0.03340767323970795
Epoch: 931 0.060180943459272385
Epoch: 931 0.0374356247484684
Epoch: 931 0.04383213445544243
Epoch: 931 0.017530979588627815
Epoch: 932 0.0329834520816803
Epoch: 932 0.034813664853572845
Epoch: 932 0.02543986216187477
Epoch: 932 0.032671116292476654
Epoch: 932 0.059085533022880554
Epoch: 932 0.03744390606880188
Epoch: 932 0.04431155323982239
Epoch: 932 0.01664172299206257
Epoch: 933 0.03262048587203026
Epoch: 933 0.03424374759197235
Epoch: 933 0.025230489671230316
Epoch: 933 0.03267524391412735
E

Epoch: 968 0.03967919200658798
Epoch: 968 0.01471567153930664
Epoch: 969 0.029340051114559174
Epoch: 969 0.03067385032773018
Epoch: 969 0.022650355473160744
Epoch: 969 0.029178325086832047
Epoch: 969 0.05355037376284599
Epoch: 969 0.03307579457759857
Epoch: 969 0.03909861668944359
Epoch: 969 0.014922495000064373
Epoch: 970 0.02906115911900997
Epoch: 970 0.030649365857243538
Epoch: 970 0.02267909236252308
Epoch: 970 0.029427312314510345
Epoch: 970 0.05357423797249794
Epoch: 970 0.033164557069540024
Epoch: 970 0.038982562720775604
Epoch: 970 0.01528973039239645
Epoch: 971 0.02938729338347912
Epoch: 971 0.03076176345348358
Epoch: 971 0.02252478152513504
Epoch: 971 0.028988724574446678
Epoch: 971 0.052879780530929565
Epoch: 971 0.033186618238687515
Epoch: 971 0.03940032422542572
Epoch: 971 0.014623324386775494
Epoch: 972 0.029162166640162468
Epoch: 972 0.0303221195936203
Epoch: 972 0.022475766018033028
Epoch: 972 0.028857102617621422
Epoch: 972 0.053433746099472046
Epoch: 972 0.03289579972

验证模型

In [61]:
testX = torch.Tensor([binary_encode(i,NUM_DIGITS) for i in range(0,101)])
if torch.cuda.is_available():
    testX = testX.cuda()
with torch.no_grad():
    testY = model(testX)

In [62]:
testY.max(1) #取第一个维度的max值

torch.return_types.max(
values=tensor([3.1277, 9.6237, 5.3191, 2.2467, 6.5294, 7.2168, 6.4853, 6.4700, 4.5836,
        4.4699, 4.6022, 6.7719, 4.1672, 6.0967, 4.0608, 6.0448, 6.6989, 3.5279,
        6.2557, 4.6073, 7.1840, 5.7346, 5.5618, 4.8215, 6.2997, 8.7942, 6.3264,
        8.9725, 5.9726, 6.6463, 8.8321, 4.7929, 5.2827, 0.7173, 4.6207, 3.8056,
        4.8275, 6.3220, 5.1120, 3.1351, 5.5605, 8.0057, 4.7746, 6.2381, 6.4671,
        8.6250, 5.9875, 5.8738, 4.7537, 5.1554, 4.5040, 4.8562, 5.8072, 4.2601,
        5.9359, 6.8937, 7.7937, 7.0684, 5.6630, 6.7015, 9.6454, 6.0673, 4.7424,
        6.1758, 7.7268, 6.2026, 5.1649, 7.0467, 4.3943, 3.0211, 8.0006, 8.6534,
        4.0150, 7.6690, 5.7596, 6.7476, 3.7157, 5.7205, 5.0650, 3.7188, 6.4641,
        6.6022, 5.4313, 4.9547, 2.5293, 6.7983, 5.1593, 3.6968, 7.2434, 6.3984,
        8.9741, 5.0277, 6.5874, 2.8553, 3.4507, 5.4124, 3.9947, 7.9334, 4.7874,
        1.9687, 6.2458]),
indices=tensor([3, 0, 0, 1, 0, 2, 1, 0, 0, 1, 2, 0, 1, 0, 0, 3,

In [63]:
predictions = zip(range(0,101),testY.max(1)[1].cpu().tolist())
print([fizz_buzz_decode(i,x) for i,x in predictions])

['fizzbuzz', '1', '2', 'fizz', '4', 'buzz', 'fizz', '7', '8', 'fizz', 'buzz', '11', 'fizz', '13', '14', 'fizzbuzz', '16', 'buzz', 'fizz', 'buzz', 'buzz', 'fizz', '22', '23', 'fizz', 'buzz', '26', 'fizz', '28', '29', 'fizzbuzz', '31', '32', '33', '34', '35', 'fizz', '37', '38', 'fizz', 'buzz', '41', 'fizz', '43', '44', 'fizzbuzz', '46', '47', 'fizz', 'buzz', 'buzz', 'fizz', '52', '53', 'fizz', 'buzz', '56', 'fizz', '58', '59', 'fizzbuzz', '61', '62', 'fizz', '64', 'buzz', 'fizz', '67', '68', '69', 'buzz', '71', 'fizz', '73', '74', 'fizzbuzz', '76', '77', 'fizz', '79', 'buzz', 'fizz', '82', '83', '84', 'buzz', '86', 'fizz', '88', '89', 'fizzbuzz', '91', '92', '93', '94', 'buzz', 'fizz', '97', '98', 'fizz', 'buzz']


### 词向量

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as tud

from collections import Counter
import numpy as np
import pandas as pd
import random
import math
import re
import codecs

import scipy
import sklearn
from sklearn.metrics.pairwise import cosine_similarity

In [4]:
USE_CUDA = torch.cuda.is_available()

random.seed(1)
np.random.seed(1)
torch.manual_seed(1)
if USE_CUDA:
    torch.cuda.manual_seed(1)

In [5]:
#设定一些hyper parameters
C = 3 #中心词周围的单词
K = 100 #负单词
NUM_EPOCH = 2
MAX_VOCAB_SIZE = 30000
BATCH_SIZE = 128
LEARNING_RATE = 0.2
EMBEDDING_SIZE = 100 #词向量维度

In [6]:
# def word_tokenize(doc):
#     doc = re.sub(r'[a-zA-Z0-9\s]','',doc,re.L)
#     doc = doc.strip()
#     return doc

In [7]:
with codecs.open("./data/text8.train.txt","r","utf-8") as fin:
    text = fin.read()
text = text.split()

In [8]:
vocab = dict(Counter(text).most_common(MAX_VOCAB_SIZE-1))
#<unk>表示不常用的词
vocab["<unk>"] = len(text) - np.sum(list(vocab.values()))

#取出字典中的词 
idx_to_word = [word for word in vocab.keys()]
#为字典中的词标注下标
word_to_idx = {x:i for i,x in enumerate(idx_to_word)}

In [9]:
list(word_to_idx.items())[-10:]

[('arrowhead', 29990),
 ('detainee', 29991),
 ('gabbro', 29992),
 ('hyperinsulinism', 29993),
 ('lantau', 29994),
 ('landsmannschaft', 29995),
 ('symmes', 29996),
 ('taif', 29997),
 ('meine', 29998),
 ('<unk>', 29999)]

In [13]:
#计算每个词的Frequency
word_counts = np.array([count for count in vocab.values()],dtype=np.float32)
word_freqs = word_counts / np.sum(word_counts)
word_freqs = word_freqs ** (3./4.)
word_freqs = word_freqs / np.sum(word_freqs)
VOCAB_SIZE = len(idx_to_word)
word_freqs

array([1.6247008e-02, 1.0514009e-02, 8.0499463e-03, ..., 5.0115582e-06,
       5.0115582e-06, 1.1669193e-02], dtype=float32)

### 利用tud.Dataset操作数据集
![QQ截图20200227113052.png](.\image\QQ截图20200227113052.png)

In [14]:
class WordEmbeddingDataset(tud.Dataset):
    def __init__(self,text,word_to_idx,idx_to_word,word_freqs,word_counts):
        super(WordEmbeddingDataset,self).__init__()
        self.text_encoded = [word_to_idx.get(word,VOCAB_SIZE-1) for word in text]
        self.text_encoded = torch.Tensor(self.text_encoded).long()
        self.word_to_idx = word_to_idx
        self.idx_to_word = idx_to_word
        self.word_freqs = torch.Tensor(word_freqs)
        self.word_counts = torch.Tensor(word_counts)
        
    def __len__(self):
        #这个数据集一共有多少个item
        return len(self.text_encoded)
    def __getitem__(self,idx):
        #根据给定的index返回一个item
        center_word = self.text_encoded[idx]
        pos_indices = list(range(idx-C,idx)) + list(range(idx+1,idx+C+1))#window内单词的index
        pos_indices = [i % len(self.text_encoded) for i in pos_indices]#取余，防止超出text长度
        pos_words = self.text_encoded[pos_indices]#周围单词
        self.word_freqs = torch.tensor(self.word_freqs)
        neg_words = torch.mutinomial(self.word_freqs,K * pos_words.shape[0],replacement=True)#replacement=True：抽到重复的样本也没事
        
        return center_word,pos_words,neg_words

In [15]:
dataset = WordEmbeddingDataset(text,word_to_idx,idx_to_word,word_freqs,word_counts)
dataloader = tud.DataLoader(dataset,batch_size=BATCH_SIZE,shuffle=True,num_workers=0)#num_workers=4：4个线程一起工作

![QQ截图20200227140156.png](.\image\QQ截图20200227140156.png)
![QQ截图20200227140026.png](.\image\QQ截图20200227140026.png)

### 定义pytorch模型
- 模型一般需要训练若干个epoch
- 每个epoch我们都把所有的数据分成若干个batch
- 把每个batch的输入和输出都包装成cuda tensor
- forward pass通过输入的句子预测每个单词的下一个单词
- 用模型的预测和正确的下一个单词计算cross entropy loss
- 清空模型当前gradient
- backward pass
- 更新模型参数
- 每隔一定的iteration输出模型再当前iteration的loss，以及在验证数据集上做模型的评估
![QQ截图20200227113143.png](.\image\QQ截图20200227113143.png)

In [4]:
class EmbeddingModel(nn.Module):
    def __init__(self,vocab_size,embed_size):
        super(EmbeddingModel,self).__init__()
        
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        
        #经验写法
        initrange = 0.5 / self.embed_size
        
        self.in_embed = nn.Embedding(self.vocab_size,self.embed_size)
        self.in_embed.weight.data.uniform_(-initrange,initrange)
        self.out_embed = nn.Embedding(self.vocab_size,self.embed_size)
        self.out_embed.weight.data.uniform_(-initrange,initrange)
        
    def forward(self,input_labels,pos_labels,neg_labels):
        #input_labels：[batch_size]
        #pos_labels:[batch_size,(window_size *2)]
        #neg_labels:[batch_size,(window_size *2 *k)]
        input_embedding = self.in_embed(input_labels)#[batch_size,embed_size]
        pos_embedding = self.out_embed(pos_labels)#[batch_size,(window_size*2),embed_size]
        neg_embedding = self.out_embed(neg_labels)#[batch_size,(window_size*2*k),embed_size]
        
        input_embedding = input_embedding.unsqueeze(2)#升维，变成:[batch_size,embed_size,1]
        pos_dot = torch.bmm(pos_embedding,input_embedding).squeeze(2) #[batch_size,(window_size*2)]
        neg_dot = torch.bmm(neg_embedding,-input_embedding).squeeze(2) #[batch_size,(window_size*2)]
        
        log_pos = F.logsigmoid(pos_dot).sum(1) #将第二维的loss加和
        log_neg = F.logsigmoid(neg_dot).sum(1)
        
        loss = log_pos + log_neg #这里直接用数学公式定义了loss，因此再训练模型的时候就不需要再定义loss了
        
        return -loss
    
    def input_embeddings(self):
        return self.in_embed.weight.data.cpu().numpy()

In [160]:
model = EmbeddingModel(VOCAB_SIZE,EMBEDDING_SIZE)
if USE_CUDA:
    model = model.cuda()

In [5]:
optimizer = torch.optim.SGD(model.parameters(),lr=LEARNING_RATE)
for e in range(NUM_EPOCH):
    for i,(input_labels,pos_labels,neg_labels) in enumerate(dataloader):
        input_labels = input_labels.long()
        pos_labels = pos_labels.long()
        neg_labels = neg_labels.long()
        if USE_CUDA:
            input_labels = input_labels.cuda()
            pos_labels = pos_labels.cuda()
            neg_labels = neg_labels.cuda()
        
        optimizer.zero_grad()
        loss = model(input_labels,pos_labels,neg_labels).mean()
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print("epoch:",e,"iteration:",i.loss.item())

### 评估

In [ ]:
def evaluate(filename,embedding_weights):
    if filename.endswith(".csv"):
        data = pd.read_csv(filename,seq=",")
    else:
        data = pd.read_csv(filename,seq="\t")
    human_similarity = []
    model_similarity = []
    for i in data.iloc[:,0:2].index:
        word1,word2 = data.iloc[i,0],data.iloc[i,1]
        if word1 not in word_to_idx or word2 not in word_to_idx:
            continue
        else:
            word1_idx,word2_idx = word_to_idx[word1],word_to_idx[word2]
            word1_embed,word2_embed = embedding_weights[[word1_idx]],embedding_weights[[word2_idx]]
            model_similarity.append(float(cosine_similarity(word1_embed,word2_embed)))
            human_similarity.append(float(data.iloc[i,2]))
    return scipy.stats.spearmanr(human_similarity,model_similarity)
def find_nearest(word):
    index = word_to_idx[word]
    embedding = embedding_weights[index]
    cos_dis = np.array([scipy.spatial.sidtance.cosine(e,embedding) for e in embedding_weights])
    return [idx_to_word[i] for i in cos_dis.argsort()[:10]]

![evaluate1](.\image\evaluate1.png)
![QQ截图20200226161903.png](.\image\QQ截图20200226161903.png)